In [1]:
import numpy as np

from collections import Counter
from itertools import islice
import re
from pprint import pprint

import nltk
from nltk.collocations import *
from nltk import ngrams

# nltk.download('punkt')

In [2]:
def n_gram_counter(file_data, n):
    counts = Counter()
    tokens = nltk.word_tokenize(file_data)
    ngram = ngrams(tokens, n)
    for n_gram in ngram :
        counts[n_gram] += 1
    return counts

def context_counts(context, n_gram_counter):
    n = len(context)
    return np.array([[item[0][n], item[1]] 
                     for item in n_gram_counter.items() if "".join(item[0][0:n]) == "".join(context)])

def genrate_word_list(file_data) :
    tokens = nltk.word_tokenize(file_data)
    ngram = ngrams(tokens, n=1)
    return [word[0] for word in ngram]

In [3]:
train_file = "Verne.5semaines.en"

with open("data/" + train_file, "r") as text_file :
    train_data = ''.join(text_file.read().split('\n'))

In [5]:
def draw_table(customer_list, d, theta):
    customer_counts = list(map(int, customer_list[:,1]))
    # TODO : Adapt the algorithm so that more than one table can hold a word
    # TODO Adapt the table count size
    table_counts_per_word = list(np.ones(len(customer_counts)))

    nb_tables = len(table_counts_per_word)
    nb_customers = sum(customer_counts)

    old_tables_probabilities = [(customer_counts[i] - d *table_counts_per_word[i] ) / (nb_customers + theta)
                                    for i in range(nb_tables)] 
    new_table_probability = [(theta + d * nb_tables) / (nb_customers + theta)]
    all_probabilities = old_tables_probabilities + new_table_probability
    
    assert np.sum(all_probabilities) == 1
    picked_table = np.argmax(np.random.multinomial(n=1, pvals=all_probabilities))
    return picked_table

In [64]:
def draw_word(context, n_gram_counters, d, theta, word_list) :
    pitman_process_level = len(context)
    if pitman_process_level > 0 :
        context_word_counts = context_counts(context, n_gram_counters[pitman_process_level])
    else :
        context_word_counts = np.array([item for item in n_gram_counters[0].items()])
    
    nb_words = len(word_list)
    nb_tables = context_word_counts.shape[0]
        
    if len(context) == 0 :
        table = draw_table(context_word_counts, d, theta)
        
        # If the table is a new table
        if table == nb_tables : 
            customer = np.random.choice(word_list)
        else :
            # TODO : Check the chosen word
            customer = context_word_counts[table, 0]
    
    else :
        table = draw_table(context_word_counts, d, theta)
        
        # If the table is a new table
        if table == nb_tables : 
            del context[0]
            customer = draw_word(context, n_gram_counters, d, theta, word_list)
        else :
            # TODO : Check the chosen word
            customer = context_word_counts[table, 0]

    return customer[0]

In [95]:
context = ["is", "the"]
d = 0.5
theta = 1
word_list = genrate_word_list(train_data)
n_gram_counters = [n_gram_counter(train_data, n=n) for n in range(1, 4)]

draw_word(context, n_gram_counters, d, theta, word_list)

'proved'